# Prefect Hello World

In [1]:
import os
os.environ['PREFECT__FLOWS__CHECKPOINTING'] = 'true'

from loguru import logger
import prefect
from prefect import task, Task, Flow, Parameter
from prefect.engine.results import LocalResult
import datetime as dt
from time import sleep

print(prefect.config.flows.checkpointing)

True


## Basic flow

In [2]:
@task
def t1():
    logger.info('t1 input: -')
    return 2.

@task
def t2(x):
    logger.info('t2 input: {}'.format(x))
    return x+1.

@task
def t3(x):
    logger.info('t3 input: {}'.format(x))
    return x+1.5

@task 
def t4(x, y):
    logger.info('t3 input: {} {}'.format(x, y))
    return x+y

with Flow('big_flow') as flow:
    r1 = t1()
    r2 = t2(r1)
    r3 = t3(r2)
    r4 = t4(r2, r3)

state = flow.run()
print('Output: ', state.result[r4].result)

[2020-10-06 08:53:43] INFO - prefect.FlowRunner | Beginning Flow run for 'big_flow'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't1': Starting task run...


2020-10-06 15:53:43.071 | INFO     | __main__:t1:3 - t1 input: -


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't1': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't2': Starting task run...


2020-10-06 15:53:43.105 | INFO     | __main__:t2:8 - t2 input: 2.0


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't3': Starting task run...


2020-10-06 15:53:43.137 | INFO     | __main__:t3:13 - t3 input: 3.0


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't3': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't4': Starting task run...


2020-10-06 15:53:43.170 | INFO     | __main__:t4:18 - t3 input: 3.0 4.5


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't4': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
Output:  7.5


## Extending an existing flow

In [3]:
with Flow('small_flow') as flow1:
    r1 = t1()
    r2 = t2(r1)

with flow1.copy() as flow2:
    flow2.name = 'extended_flow'
    r3 = t3(r2)
    r4 = t4(r2, r3)

state = flow1.run()
print('Output: ', state.result[r2].result)

print()
state = flow2.run()
print('Output: ', state.result[r4].result)

[2020-10-06 08:53:43] INFO - prefect.FlowRunner | Beginning Flow run for 'small_flow'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't1': Starting task run...


2020-10-06 15:53:43.231 | INFO     | __main__:t1:3 - t1 input: -


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't1': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't2': Starting task run...


2020-10-06 15:53:43.266 | INFO     | __main__:t2:8 - t2 input: 2.0


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
Output:  3.0

[2020-10-06 08:53:43] INFO - prefect.FlowRunner | Beginning Flow run for 'extended_flow'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't1': Starting task run...


...

[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't3': Starting task run...


2020-10-06 15:53:43.384 | INFO     | __main__:t3:13 - t3 input: 3.0


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't3': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't4': Starting task run...


2020-10-06 15:53:43.419 | INFO     | __main__:t4:18 - t3 input: 3.0 4.5


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't4': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
Output:  7.5


In [4]:
with Flow('small_flow') as flow1:
    r1 = t1()
    r2 = t2(r1)

with Flow('extended_flow') as flow2:
    flow2.update(flow1)
    r3 = t3(r2)
    r4 = t4(r2, r3)

state = flow1.run()
print('Output: ', state.result[r2].result)

print()
state = flow2.run()
print('Output: ', state.result[r4].result)

[2020-10-06 08:53:43] INFO - prefect.FlowRunner | Beginning Flow run for 'small_flow'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't1': Starting task run...


2020-10-06 15:53:43.478 | INFO     | __main__:t1:3 - t1 input: -


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't1': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't2': Starting task run...


2020-10-06 15:53:43.510 | INFO     | __main__:t2:8 - t2 input: 2.0


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
Output:  3.0

[2020-10-06 08:53:43] INFO - prefect.FlowRunner | Beginning Flow run for 'extended_flow'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't1': Starting task run...


...

[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't3': Starting task run...


2020-10-06 15:53:43.620 | INFO     | __main__:t3:13 - t3 input: 3.0


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't3': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't4': Starting task run...


2020-10-06 15:53:43.655 | INFO     | __main__:t4:18 - t3 input: 3.0 4.5


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't4': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
Output:  7.5


## Parametrized flow

In [5]:
with Flow('param_flow') as flow:
    y = Parameter('y')
    scaling = Parameter('scaling', 1.)
    r1 = t1()
    r2 = t2(r1)
    r4 = scaling*t4(r2, y)

    
state = flow.run(dict(y=4.5))
print('Output: ', state.result[r4].result)
print()
state = flow.run(dict(y=4.5, scaling=2.))
print('Output: ', state.result[r4].result)

[2020-10-06 08:53:43] INFO - prefect.FlowRunner | Beginning Flow run for 'param_flow'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't1': Starting task run...


2020-10-06 15:53:43.712 | INFO     | __main__:t1:3 - t1 input: -


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't1': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 'y': Starting task run...
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 'y': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't2': Starting task run...


2020-10-06 15:53:43.790 | INFO     | __main__:t2:8 - t2 input: 2.0


[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 'scaling': Starting task run...
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 'scaling': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't4': Starting task run...


...

[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 't1': finished task run for task with final state: 'Success'
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 'y': Starting task run...
[2020-10-06 08:53:43] INFO - prefect.TaskRunner | Task 'y': finished task run for task with final state: 'Success'
[2020-10-06 08:53:44] INFO - prefect.TaskRunner | Task 't2': Starting task run...


2020-10-06 15:53:44.006 | INFO     | __main__:t2:8 - t2 input: 2.0


[2020-10-06 08:53:44] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Success'
[2020-10-06 08:53:44] INFO - prefect.TaskRunner | Task 'scaling': Starting task run...
[2020-10-06 08:53:44] INFO - prefect.TaskRunner | Task 'scaling': finished task run for task with final state: 'Success'
[2020-10-06 08:53:44] INFO - prefect.TaskRunner | Task 't4': Starting task run...


2020-10-06 15:53:44.072 | INFO     | __main__:t4:18 - t3 input: 3.0 4.5


[2020-10-06 08:53:44] INFO - prefect.TaskRunner | Task 't4': finished task run for task with final state: 'Success'
[2020-10-06 08:53:44] INFO - prefect.TaskRunner | Task 'Mul': Starting task run...
[2020-10-06 08:53:44] INFO - prefect.TaskRunner | Task 'Mul': finished task run for task with final state: 'Success'
[2020-10-06 08:53:44] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
Output:  15.0


## Cached flow

In [6]:
@task(cache_for=dt.timedelta(hours=1))
def t1():
    sleep(2)
    logger.info('t1++ input: -')
    return 2.

@task(cache_for=dt.timedelta(hours=1))
def t2(x):
    sleep(2)
    logger.info('t2++ input: {}'.format(x))
    return x+1.


with Flow('cached_flow') as flow:
    r1 = t1()
    r2 = t2(r1)

state = flow.run()
print('Output: ', state.result[r2].result)

print()
state = flow.run()
print('Output: ', state.result[r2].result)

[2020-10-06 08:53:44] INFO - prefect.FlowRunner | Beginning Flow run for 'cached_flow'
[2020-10-06 08:53:44] INFO - prefect.TaskRunner | Task 't1': Starting task run...
[2020-10-06 08:53:44] WARNING - prefect.TaskRunner | Task 't1': can't use cache because it is now invalid


2020-10-06 15:53:46.157 | INFO     | __main__:t1:4 - t1++ input: -


[2020-10-06 08:53:46] INFO - prefect.TaskRunner | Task 't1': finished task run for task with final state: 'Cached'
[2020-10-06 08:53:46] INFO - prefect.TaskRunner | Task 't2': Starting task run...
[2020-10-06 08:53:46] WARNING - prefect.TaskRunner | Task 't2': can't use cache because it is now invalid


2020-10-06 15:53:48.201 | INFO     | __main__:t2:10 - t2++ input: 2.0


[2020-10-06 08:53:48] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Cached'
[2020-10-06 08:53:48] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
Output:  3.0

[2020-10-06 08:53:48] INFO - prefect.FlowRunner | Beginning Flow run for 'cached_flow'
[2020-10-06 08:53:48] INFO - prefect.TaskRunner | Task 't1': Starting task run...
[2020-10-06 08:53:48] INFO - prefect.TaskRunner | Task 't1': finished task run for task with final state: 'Cached'
[2020-10-06 08:53:48] INFO - prefect.TaskRunner | Task 't2': Starting task run...
[2020-10-06 08:53:48] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Cached'
[2020-10-06 08:53:48] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
Output:  3.0


In [7]:
@task(checkpoint=True, result=LocalResult(dir='cache'), target="{flow_name}_{task_name}")
def t1():
    sleep(2)
    logger.info('t1++ input: -')
    return 2.

@task(checkpoint=True, result=LocalResult(dir='cache'), target="{flow_name}_{task_name}")
def t2(x):
    sleep(2)
    logger.info('t2++ input: {}'.format(x))
    return x+1.


with Flow('persisted_flow') as flow:
    r1 = t1()
    r2 = t2(r1)

state = flow.run()
print('Output: ', state.result[r2].result)

print()
state = flow.run()
print('Output: ', state.result[r2].result)

[2020-10-06 08:53:48] INFO - prefect.FlowRunner | Beginning Flow run for 'persisted_flow'
[2020-10-06 08:53:48] INFO - prefect.TaskRunner | Task 't1': Starting task run...


2020-10-06 15:53:50.368 | INFO     | __main__:t1:4 - t1++ input: -


[2020-10-06 08:53:50] INFO - prefect.TaskRunner | Task 't1': finished task run for task with final state: 'Success'
[2020-10-06 08:53:50] INFO - prefect.TaskRunner | Task 't2': Starting task run...


2020-10-06 15:53:52.404 | INFO     | __main__:t2:10 - t2++ input: 2.0


[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Success'
[2020-10-06 08:53:52] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
Output:  3.0

[2020-10-06 08:53:52] INFO - prefect.FlowRunner | Beginning Flow run for 'persisted_flow'
[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 't1': Starting task run...
[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 't1': finished task run for task with final state: 'Cached'
[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 't2': Starting task run...
[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Cached'
[2020-10-06 08:53:52] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
Output:  3.0


In [8]:
@task(checkpoint=True, result=LocalResult(dir='cache'), target="{flow_name}_{task_name}_{parameters}")
def t1():
    sleep(2)
    logger.info('t1++ input: -')
    return 2.

@task(checkpoint=True, result=LocalResult(dir='cache'), target="{flow_name}_{task_name}_{parameters}")
def t2(x, y):
    sleep(2)
    logger.info('t2++ input: {}'.format(x))
    return x+y


with Flow('parametrized_persisted_flow') as flow:
    y = Parameter('y')
    r1 = t1()
    r2 = t2(r1, y)

state = flow.run(dict(y=1.))
print('Output: ', state.result[r2].result)

print()
state = flow.run(dict(y=1.))
print('Output: ', state.result[r2].result)

print()
state = flow.run(dict(y=2.))
print('Output: ', state.result[r2].result)

[2020-10-06 08:53:52] INFO - prefect.FlowRunner | Beginning Flow run for 'parametrized_persisted_flow'
[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 't1': Starting task run...
[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 't1': finished task run for task with final state: 'Cached'
[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 'y': Starting task run...
[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 'y': finished task run for task with final state: 'Success'
[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 't2': Starting task run...
[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 't2': finished task run for task with final state: 'Cached'
[2020-10-06 08:53:52] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
Output:  3.0

[2020-10-06 08:53:52] INFO - prefect.FlowRunner | Beginning Flow run for 'parametrized_persisted_flow'
[2020-10-06 08:53:52] INFO - prefect.TaskRunner | Task 't1': Starting task run...
[2020-10-0